In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
from shapely.ops import unary_union, polygonize

# 1. 여의도 도로 네트워크 데이터 가져오기
place_name = "Yeouido-dong, Seoul, South Korea"
# 여의도 지역의 도로 네트워크를 드라이브 타입으로 가져옵니다.
graph = ox.graph_from_place(place_name, network_type="drive")

# 도로 네트워크에서 노드와 엣지 데이터를 추출
nodes, edges = ox.graph_to_gdfs(graph)

# 도로 네트워크를 기반으로 폴리곤(구역) 생성
lines = edges['geometry'].values
merged_lines = unary_union(lines)  # 도로를 하나로 병합하여 경계선을 만듭니다.
polygons = list(polygonize(merged_lines))  # 병합된 도로를 기준으로 폴리곤 생성

# 폴리곤을 GeoDataFrame으로 변환
polygons_gdf = gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")

# 2. 인구 데이터 불러오기
# 파일 경로와 범위 설정
base_path = "/content/drive/MyDrive/여의도_불꽃축제_데이터/인구종합/"
file_template = "2023-10-07_{:02d}_인구종합.csv"

# 시간별 데이터를 하나의 데이터프레임으로 병합
all_data = pd.DataFrame()
for hour in range(24):  # 00시부터 23시까지 반복하여 데이터 읽기
    file_path = base_path + file_template.format(hour)
    try:
        # 파일 읽기
        df = pd.read_csv(file_path)
        all_data = pd.concat([all_data, df])  # 데이터 병합
    except FileNotFoundError:
        print(f"파일이 없습니다: {file_path}")
        continue

# 격자별 인구수 총합 계산
aggregated_population = all_data.groupby(['X_COORD', 'Y_COORD'])['POPL_CNT'].sum().reset_index()

# GeoDataFrame 변환
geometry = [Point(xy) for xy in zip(aggregated_population.X_COORD, aggregated_population.Y_COORD)]
population_gdf = gpd.GeoDataFrame(aggregated_population, geometry=geometry, crs="EPSG:4326")

# 3. 폴리곤별 인구 합산
polygons_gdf["POPULATION"] = 0  # 폴리곤별 인구수 저장 열 초기화
for idx, polygon in polygons_gdf.iterrows():
    # 각 폴리곤 내 포함된 인구 데이터 필터링
    mask = population_gdf.within(polygon.geometry)
    total_population = population_gdf.loc[mask, "POPL_CNT"].sum()
    polygons_gdf.at[idx, "POPULATION"] = total_population  # 해당 폴리곤에 포함된 총 인구수 계산

# 4. Folium 지도 시각화
m = folium.Map(location=[37.526260, 126.926979], zoom_start=15)

# 컬러맵 설정 (붉은색 계열, 인구 밀도가 높을수록 붉은 색이 강해짐)
max_population = polygons_gdf["POPULATION"].max()
colormap = folium.LinearColormap(colors=["white", "red"], vmin=0, vmax=max_population)
colormap.caption = "Population Density"  # 컬러맵에 제목 추가

# 각 폴리곤을 지도에 시각화
for _, row in polygons_gdf.iterrows():
    if row.POPULATION > 0:  # 인구수가 0보다 큰 폴리곤만 표시
        folium.Polygon(
            locations=[(point[1], point[0]) for point in row.geometry.exterior.coords],
            color=None,
            fill=True,
            fill_color=colormap(row["POPULATION"]),  # 인구 수에 비례하여 색상 지정
            fill_opacity=0.7,
            popup=f"총 인구수: {row['POPULATION']}"  # 팝업에 인구수 표시
        ).add_to(m)

# 컬러맵 추가
m.add_child(colormap)

# 지도 저장
map_file_path = "/content/drive/MyDrive/여의도_인구밀집도_구역별_지도.html"
m.save(map_file_path)
print(f"지도 파일이 저장되었습니다: {map_file_path}")

# 지도 표시
m